# Modelos de IA para previsão de notas acadêmicas

Com base em um conjunto de dados de notas acadêmicas, vamos criar modelos de aprendizado de máquina para prever as notas dos alunos e alertar sobre riscos de possíveis dificultades como prova final e reprovações.

## 1. Tratamento dos Dados

Primeiramente, precisamos adaptar os dados para o formato adequado para o treinamento dos modelos. No conjunto de dados `notas.csv`, Nos interessamos apenas nas colunas `AlunoID` que corresponde ao identificador do aluno (precisamos ter seu perfil salvo para melhor análise da IA), `Disciplina` que é a disciplina em questão e as colunas `Unidade X` sendo X o número da unidade.


Vamos começar importando as bibliotecas, abrir o CSV e convertendo as notas pra float:


In [ ]:
import pandas as pd


# Carrega os dados do arquivo CSV
notas = pd.read_csv('dados/notas.csv')

# Converter as colunas de notas para o tipo float
for unidade in ['Unidade 1', 'Unidade 2', 'Unidade 3', 'Unidade 4', 'Unidade 5']:
    notas[unidade] = notas[unidade].str.replace(',', '.').astype(float)

Após isso, vamos criar uma coluna `Media` que será a média das notas de cada aluno em cada disciplina. Não vamos considerar a nota da prova final para o cálculo da média, pois ela é uma nota de recuperação e vamos considerar apenas as unidades que tenham notas pois, se estiver em branco, significa que a disciplina não tem nota naquela unidade.


In [ ]:
# Calcular a média das notas, sem prova final
notas['Media'] = notas[['Unidade 1', 'Unidade 2', 'Unidade 3', 'Unidade 4', 'Unidade 5']].mean(axis=1, skipna=True)


Com a coluna `Media` criada, agora podemos pivotear os dados. `AlunoID` será o índice, `Disciplina` será as colunas e as notas serão os valores. Isso nos dará um DataFrame onde cada linha representa um aluno e cada coluna uma disciplina, com as notas correspondentes. Um aluno pode ter cursado uma disciplina mais de uma vez em caso de reprovação, por isso pode gerar conflitos no pivotamento. Para resolver isso, vamos usar a menor nota, pois é a nota que o aluno pode ter reprovado/ficado de prova final.


In [ ]:
notas_pivoteadas = notas.pivot_table(
    index='AlunoID',
    columns='Disciplina',
    values='Media',
    aggfunc='min'
)

O resultado ficando assim:


In [ ]:
notas_pivoteadas.head()

Agora, vamos salvar o DataFrame resultante em um arquivo CSV.


In [ ]:
notas_pivoteadas.to_csv('dados/notas_pivoteadas.csv', index=False)

## 2. Treinamenro do Modelo

In [ ]:
import pandas as pd

df = pd.read_csv('dados/notas_pivoteadas.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

x = df[
[    'PROGRAMAÇÃO ESTRUTURADA',
    'INTRODUÇÃO À LÓGICA',
    'INGLÊS TÉCNICO E CIENTÍFICO',
    'INFORMÁTICA E SOCIEDADE',
    'CÁLCULO DIFERENCIAL E INTEGRAL I',
    'INTRODUÇÃO À METODOLOGIA CIENTÍFICA']
]
y = df[
[    'CÁLCULO DIFERENCIAL E INTEGRAL II',
    'CIRCUITOS DIGITAIS',
    'ESTRUTURAS DE DADOS',
    'LABORATÓRIO DE PROGRAMAÇÃO',
    'MATEMÁTICA DISCRETA',
    'PROGRAMAÇÃO ORIENTADA A OBJETOS',]
]

dados = pd.concat([x, y], axis=1).dropna()
x = dados[x.columns]
y = dados[y.columns]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
mean_absolute_error(y_test, y_pred)


0.630822222222225

In [ ]:
# ver precisao e notas reais
for i, col in enumerate(y.columns):
    print(f'Precisão para {col}: {mean_absolute_error(y_test[col], y_pred[:, i])}')
    print(f'Notas reais: {y_test[col].values[:5]}')
    print(f'Notas previ: {y_pred[:, i][:5]}')
    print()

Precisão para CÁLCULO DIFERENCIAL E INTEGRAL II: 0.6391333333333356
Notas reais: [9.26666667 8.13333333 8.5        6.23333333 8.13333333]
Notas previ: [8.908      8.13333333 8.62       8.95033333 8.13333333]

Precisão para CIRCUITOS DIGITAIS: 0.3384000000000018
Notas reais: [9.03333333 7.5        9.83333333 8.53333333 7.5       ]
Notas previ: [9.148      7.5        8.814      9.09133333 7.5       ]

Precisão para ESTRUTURAS DE DADOS: 0.6616666666666697
Notas reais: [8.9        8.03333333 8.93333333 5.96666667 8.03333333]
Notas previ: [9.00433333 8.03333333 8.48233333 8.71966667 8.03333333]

Precisão para LABORATÓRIO DE PROGRAMAÇÃO: 0.6286666666666723
Notas reais: [9.43333333 9.2        9.43333333 7.8        9.2       ]
Notas previ: [8.746      9.2        7.628      8.45066667 9.2       ]

Precisão para MATEMÁTICA DISCRETA: 0.06633333333333233
Notas reais: [7.33333333 7.         7.33333333 7.66666667 7.        ]
Notas previ: [7.335      7.         7.31166667 7.35833333 7.        ]

Prec

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.multioutput import MultiOutputRegressor

# ...carregue e prepare x e y como já está no seu código...

model = MultiOutputRegressor(RandomForestRegressor())

# KFold com 5 divisões
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# cross_val_score retorna scores negativos para erro, por isso usamos 'neg_mean_absolute_error'
scores = cross_val_score(model, x, y, cv=kf, scoring='neg_mean_absolute_error')

# Como os scores são negativos, inverta o sinal para obter o MAE positivo
mae_scores = -scores

print(f'MAE em cada fold: {mae_scores}')
print(f'MAE médio: {mae_scores.mean():.2f}')

MAE em cada fold: [0.67863333 0.66366667 0.07452778 0.42170833 0.41248611]
MAE médio: 0.45


In [ ]:
x = df[[
    'PROGRAMAÇÃO ESTRUTURADA',
    'INTRODUÇÃO À LÓGICA',
    'INGLÊS TÉCNICO E CIENTÍFICO',
    'INFORMÁTICA E SOCIEDADE',
    'CÁLCULO DIFERENCIAL E INTEGRAL I',
    'INTRODUÇÃO À METODOLOGIA CIENTÍFICA',
    'CÁLCULO DIFERENCIAL E INTEGRAL II',
    'CIRCUITOS DIGITAIS',
    'ESTRUTURAS DE DADOS',
    'LABORATÓRIO DE PROGRAMAÇÃO',
    'MATEMÁTICA DISCRETA',
    'PROGRAMAÇÃO ORIENTADA A OBJETOS'
]]

y = df[[
    'ÁLGEBRA LINEAR',
    'ARQUITETURA DE COMPUTADORES',
    'PROBABILIDADE E ESTATÍSTICA',
    'BANCO DE DADOS',
    'INTERFACE HUMANO COMPUTADOR',
    'PROCESSAMENTO DIGITAL DE IMAGENS',
]]

dados = pd.concat([x, y], axis=1).dropna()
x = dados[x.columns]
y = dados[y.columns]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
mean_absolute_error(y_test, y_pred)

0.6268555555555578

In [ ]:
# ver precisao e notas reais
for i, col in enumerate(y.columns):
    print(f'Precisão para {col}: {mean_absolute_error(y_test[col], y_pred[:, i])}')
    print(f'Notas reais: {y_test[col].values[:5]}')
    print(f'Notas previ: {y_pred[:, i][:5]}')
    print()

Precisão para ÁLGEBRA LINEAR: 0.6285333333333389
Notas reais: [7.5        8.26666667 7.16666667 7.         8.26666667]
Notas previ: [5.526      8.26666667 7.649      6.31366667 8.26666667]

Precisão para ARQUITETURA DE COMPUTADORES: 1.9384666666666668
Notas reais: [2.33333333 7.         2.33333333 7.06666667 7.        ]
Notas previ: [6.68333333 7.         6.81533333 6.20633333 7.        ]

Precisão para PROBABILIDADE E ESTATÍSTICA: 0.6644666666666673
Notas reais: [9.1        5.8        8.76666667 7.         5.8       ]
Notas previ: [8.33133333 5.8        6.446      7.233      5.8       ]

Precisão para BANCO DE DADOS: 0.12400000000000481
Notas reais: [9.66666667 9.83333333 9.66666667 9.16666667 9.83333333]
Notas previ: [9.44333333 9.83333333 9.65       9.54666667 9.83333333]

Precisão para INTERFACE HUMANO COMPUTADOR: 0.29480000000000073
Notas reais: [9.7        8.03333333 9.66666667 8.5        8.03333333]
Notas previ: [9.26233333 8.03333333 9.31033333 9.18       8.03333333]

Precisão 